In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import xarray as xr
import hvplot.xarray
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import pandas as pd

import xopr.opr_access as xopr

In [ ]:
# Establish an OPR session
# You'll probably want to set a cache directory if you're running this locally to speed
# up subsequent requests. You can do other things like customize the STAC API endpoint,
# but you shouldn't need to do that for most use cases.
opr = xopr.OPRConnection(cache_dir="radar_cache")

# Or you can open a connection without a cache directory (for example, if you're parallelizing
# this on a cloud cluster without persistent storage).
#opr = xopr.OPRConnection()

In [ ]:
# List the available OPR datasets
collections = opr.get_collections()
print([c['id'] for c in collections])
selected_collection = collections[1]['id']  # Select a collection for demonstration
print(f"Selected collection: {selected_collection}")

In [ ]:
# List flights in the selected collection
flights = opr.get_flights(selected_collection)
print(f"Found {len(flights)} flights in collection {selected_collection}")

In [ ]:
selected_flight = flights[3]['flight_id']  # Select a flight for demonstration
print(f"Selected flight: {selected_flight}")

frames = opr.load_flight(selected_collection, flight_id=selected_flight)

In [ ]:
# Inspect an individual frame
frames[0]

In [ ]:
# Combine the frames into a single xarray Dataset representing the flight line
flight_line = xr.concat(frames, dim='slow_time', combine_attrs='drop_conflicts')
flight_line

In [ ]:
layers = None
try:
    layers = xopr.get_layers(flight_line)
    layers[1] # Display the surface layer as an example
except Exception as e:
    layers = None
    print(f"Error fetching layers: {e}")

In [ ]:
# Plot a map of where the data was collected with a basemap on an EPSG:3031 projection
proj = ccrs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
coords = flight_line[['Longitude', 'Latitude']].to_dataframe().dropna()
(gf.ocean * gf.coastline * gv.Points([coords.values])).opts(projection=proj, aspect='equal')


In [ ]:
#stacked = flight_line.Data.rolling(slow_time=10, center=True).mean()
stacked = flight_line.Data.resample(slow_time='2s').mean()
pwr_dB = 10*np.log10(np.abs(stacked))

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4))
pwr_dB.plot.imshow(x='slow_time', cmap='gray', ax=ax)
ax.invert_yaxis()

if layers:
    layers[1]['twtt'].plot(ax=ax, x='slow_time', linestyle=':', label='Surface')
    layers[2]['twtt'].plot(ax=ax, x='slow_time', linestyle='--', label='Bed')
    ax.legend()

In [ ]:
print(opr.generate_citation(flight_line))